In [ ]:
using SpecialFunctions, IterTools, HardSphereDynamics

Aqui abajo Rosa define todos los resultados téoricos para cualquier número de dimensiones, siempre
que no haya "sombras prohibidas" (es decir, que las bolas siempre quepan unas al lado de otras).

In [1]:

function SphereArea(n)
    return 2*(pi^(n/2))/gamma(n/2)
end

function NormalizationFactor(n, balls=2)
    d = 2*n
    return SphereArea(d)/SphereArea(d-1)*(d-1)
end

function Ak(boxdims,m)
    result = 0
    for aks in subsets(boxdims, m)
        result += prod(aks)
    end
    return result
end

function AvailableVolume(n, boxdims, r)
    vol = prod((2*boxdims).^2)
    for k in 0:n
        vol -= (4^n)*(pi)^(n/2-k/2)*r^(k+n)/gamma(k/2+n/2+1)*Ak(boxdims, n-k)*(-1)^k
    end
    return vol
end

function SphereCollisionArea(n, boxdims, r)
    area = 0 
    for k in 0:n
        area += 2^(2*n+1/2)*(pi^(n/2-k/2))*(r^(k+n-1))/gamma(k/2+n/2)*Ak(boxdims, n-k)*(-1)^k
    end
    return area
end

function WallCollisionArea(n, boxdims, r, dim)
    otherboxdims = deleteat!(copy(boxdims),dim)
    area = prod((2*otherboxdims).^2)*(2*boxdims[dim])
    for k in 0:(n-1)
        area -= 2^(2*n-2)*(pi^(n/2 - k/2))*(r^(k+n))/gamma(k/2+n/2+1)*(-1)^k*Ak(otherboxdims, n-1-k)
    end
    return area
end

function SphereCollisionTime(n, boxdimsh, r)
    boxdimsa = boxdimsh/2 .- r 
    return NormalizationFactor(n)*AvailableVolume(n, boxdimsa, r)/SphereCollisionArea(n, boxdimsa, r)
end

function WallCollisionTime(n, boxdimsh, r, dim)
    boxdimsa = boxdimsh/2 .- r 
    return NormalizationFactor(n)*AvailableVolume(n, boxdimsa, r)/WallCollisionArea(n, boxdimsa, r, dim)
end
    
function HoppingCollisionTime(n, boxdimsh, r, dim)
    boxdimsa = boxdimsh/2 .- r 
    otherboxdims = deleteat!(copy(boxdimsa),dim)
    return  NormalizationFactor(n)*AvailableVolume(n, boxdimsa, r)/AvailableVolume(n-1, otherboxdims, r)/2^(2.5)/boxdimsa[dim]
end

function CollisionTimesTheo(r, n=3, boxdimsh = [1.0, 1.5, 1.75])
    wall_times = []
    hop_times = []
    for dim in 1:n
        w = WallCollisionTime(n, boxdimsh, r, dim)
        push!(wall_times, w, w)
        push!(hop_times, HoppingCollisionTime(n, boxdimsh, r, dim))
    end
    return SphereCollisionTime(n, boxdimsh, r), wall_times, hop_times
end

CollisionTimesTheo (generic function with 3 methods)

Aqui Rosa hace la numerica "hard core"

In [13]:
?AllToAll

search: AllToAll task_local_storage



No documentation found.

`HardSphereDynamics.AllToAll` is of type `UnionAll`.

# Summary

```
struct UnionAll <: Type{T}
```

# Fields

```
var  :: TypeVar
body :: Any
```

# Supertype Hierarchy

```
UnionAll <: Type{T} <: Any
```


In [2]:
using Parameters, Statistics, HardSphereDynamics, StaticArrays

function BallsRelativeSign(state)
    b1, b2 = state
    return broadcast(sign, b1.x - b2.x)
end

function mean_interval(times)
    return mean(diff(times))
end

h1 = 1.0
h2 = 1.5
h3 = 1.75
r = 0.1

function CollisionTimesSim(r, h1 = 1.0, h2 = 1.5, h3 = 1.75 ,num_collisions = 10^7)
    table = HardSphereDynamics.RectangularBox(SA[-h1/2, -h2/2, -h3/2],
                                              SA[+h1/2, +h2/2, +h3/2])

    n = 3
    spheres = 2
    fluid = HardSphereFluid{n,Float64}(table, spheres, r)
    initial_condition!(fluid.balls, table)

    # set up simulation:
    collision_type = ElasticCollision()
    flow_type = ExternalFieldFlow(SA[0.0, 0.0, 0.0])
    event_handler = AllToAll(fluid, flow_type)

    simulation =  HardSphereSimulation(
        fluid, event_handler, flow_type, collision_type);

    @unpack fluid, flow_dynamics, collision_dynamics, event_handler = simulation
    states = [deepcopy(fluid.balls)]
    times = [0.0]
    collision_types = [:none]
    partner2 = [0] 
    partner1 = [0]
    hopping_times = []
    hopping_directions = []
    for i in 1:num_collisions
        HardSphereDynamics.flow!(fluid.balls, (event_handler.next_collision_time - simulation.current_time), 
            flow_dynamics)
        simulation.current_time = event_handler.next_collision_time
        push!(collision_types, event_handler.collision_type)
        push!(partner2, event_handler.partner2)
        push!(partner1, event_handler.partner1)
        HardSphereDynamics.collide!(fluid, event_handler, collision_dynamics)
        HardSphereDynamics.find_collision!(event_handler, fluid, flow_dynamics)
        push!(states, deepcopy(fluid.balls))
        push!(times, simulation.current_time)
        prev, current = states[i:i+1]
        signprev, signcurrent = broadcast(BallsRelativeSign,[prev, current])
        if any(signprev .!= signcurrent)
            b1, b2 = states[i]
            start_time, final_time = times[i:i+1]
            hop_times = start_time .+( b1.x - b2.x)./(b2.v - b1.v)
            for (dir, time) in enumerate(hop_times)
                if all([time>=start_time, time<=final_time])
                    push!(hopping_times, time)
                    push!(hopping_directions, dir)
                end
            end
        end
    end
    wall_idx = (collision_types.== :wall_collision).&(partner1.== 1)
    wall_times = times[wall_idx];
    which_wall = partner2[wall_idx];
    wall_means = []
    for i in 1:length(fluid.box.walls)
        push!(wall_means,mean_interval(wall_times[which_wall .== i]))
    end
    hop_means = []
    for i in 1:n
        push!(hop_means, mean_interval(hopping_times[hopping_directions .== i]))
    end
    return mean_interval(times[collision_types.== :disc_collision]), wall_means, hop_means
end

CollisionTimesSim (generic function with 5 methods)

In [6]:
129*24/60*10/60

8.6

In [4]:
@time CollisionTimesSim(0.01)

128.881214 seconds (312.93 M allocations: 17.068 GiB, 28.92% gc time)


(2087.4676667653957, Any[5.70341596727526, 5.70354897032022, 8.93465853987921, 8.934912745983347, 10.434211649907011, 10.434076459882764], Any[2.016412006099097, 3.0909948732037926, 3.619374802197682])

In [ ]:
CollisionTimesTheo(0.01)

In [ ]:
rs = 0.01:0.01:0.24
theo = broadcast(CollisionTimesTheo, rs);
sim = broadcast(CollisionTimesSim, rs);

In [ ]:
using JLD

In [ ]:
nota="Se usaron las medidas de la caja h1=$h1, h2=$h2, h3=$h3, y se tomaron 10^7 eventos de la simulacion."
otranota="los radios fueron de 0.1 a 0.24 en pasos de 0.01"

onde="simulaRosa20201215-1646.jld"

save(onde, "teoricas" , theo, "numericas", sim, "radios", rs, "nota01", nota, "nota02", otranota)

In [ ]:
using Plots
scatter(rs,[s[1] for s in sim])
plot!(rs,[t[1] for t in theo])
title!("Disc collisions")
xlabel!("r")
ylabel!("mean time")

In [ ]:
scatter(rs,[s[2][1] for s in sim])
plot!(rs,[t[2][1] for t in theo])
title!("X wall collisions")
xlabel!("r")
ylabel!("mean time")

In [ ]:
scatter(rs,[s[2][3] for s in sim])
plot!(rs,[t[2][3] for t in theo])
title!("y wall collisions")
xlabel!("r")
ylabel!("mean time")

In [ ]:
scatter(rs,[s[2][5] for s in sim])
plot!(rs,[t[2][5] for t in theo])
title!("z wall collisions")
xlabel!("r")
ylabel!("mean time")

In [ ]:
scatter(rs,[s[3][1] for s in sim])
plot!(rs,[t[3][1] for t in theo])
title!("Hopping x")
xlabel!("r")
ylabel!("mean time")

In [ ]:
scatter(rs,[s[3][2] for s in sim])
plot!(rs,[t[3][2] for t in theo])
title!("Hopping y")
xlabel!("r")
ylabel!("mean time")

In [ ]:
scatter(rs,[s[3][3] for s in sim])
plot!(rs,[t[3][3] for t in theo])
title!("Hopping z")
xlabel!("r")
ylabel!("mean time")

A box with different dimensions

In [ ]:
h1 = 1
h2 = 2
h3 = 3
theo2 = broadcast(function (r)  CollisionTimesTheo(r, 3, [h1,h2,h3]) end, rs);
sim2 = broadcast(function (r) CollisionTimesSim(r, h1, h2, h3) end , rs);

In [ ]:
using Plots
scatter(rs,[s[1] for s in sim2])
plot!(rs,[t[1] for t in theo2])
title!("Disc collisions")
xlabel!("r")
ylabel!("mean time")

In [ ]:
scatter(rs,[s[2][1] for s in sim2])
plot!(rs,[t[2][1] for t in theo2])
title!("x wall collisions")
xlabel!("r")
ylabel!("mean time")

In [ ]:
scatter(rs,[s[2][3] for s in sim2])
plot!(rs,[t[2][3] for t in theo2])
title!("y wall collisions")
xlabel!("r")
ylabel!("mean time")

In [ ]:
scatter(rs,[s[2][5] for s in sim2])
plot!(rs,[t[2][5] for t in theo2])
title!("z wall collisions")
xlabel!("r")
ylabel!("mean time")

In [ ]:
scatter(rs,[s[3][1] for s in sim2])
plot!(rs,[t[3][1] for t in theo2])
title!("Hopping x")
xlabel!("r")
ylabel!("mean time")

In [ ]:
scatter(rs,[s[3][2] for s in sim])
plot!(rs,[t[3][2] for t in theo])
title!("Hopping y")
xlabel!("r")
ylabel!("mean time")

In [ ]:
scatter(rs,[s[3][3] for s in sim2])
plot!(rs,[t[3][3] for t in theo2])
title!("Hopping z")
xlabel!("r")
ylabel!("mean time")